## Model training using the sktime library

In [2]:
import os
import sys
import json
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
from timeit import default_timer as timer

from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sktime.datatypes import convert_to
from sktime.datasets import load_from_tsfile
from sktime.classification.compose import ColumnEnsembleClassifier
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.interval_based import SupervisedTimeSeriesForest
from sktime.classification.distance_based import ProximityForest
from sktime.classification.dictionary_based import MUSE

SEED=42
warnings.filterwarnings('ignore')
np.random.seed(42)

## Data preparation

In [3]:
X, y = load_from_tsfile("data/alfa-uav-fault-detection.ts", return_y=True, return_data_type="numpy3D")

X.shape, y.shape

((2792, 17, 40), (2792,))

In [4]:
X, y = shuffle(X, y, random_state=SEED)
X.shape, y.shape

((2792, 17, 40), (2792,))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, random_state=SEED, 
                                                    shuffle=True, stratify=y)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2233, 17, 40) (2233,)
(559, 17, 40) (559,)


## SupervisedTimeSeriesForest model training 

In [ ]:
%%time

estimators = []

for i in list(range(X_train.shape[1])):
    estimators.append(
        (f"STSF{i}", SupervisedTimeSeriesForest(n_estimators=5, n_jobs=4, random_state=SEED), [i])
    )

clf = ColumnEnsembleClassifier(estimators=estimators)

clf = clf.fit(X_train, y_train)

In [52]:
%%time

y_pred = clf.predict(X_test)

print("Classification report:\n")
print(classification_report(y_test, y_pred))

Classification report:

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       520
           1       1.00      0.13      0.23        39

    accuracy                           0.94       559
   macro avg       0.97      0.56      0.60       559
weighted avg       0.94      0.94      0.92       559

Wall time: 9.79 s


## WEASEL+MUSE model training

In [7]:
%%time

muse_model = MUSE(n_jobs=2, random_state=SEED)
muse_model.fit(X_train, y_train)

CPU times: total: 15min 47s
Wall time: 16min 45s


MUSE(n_jobs=2, random_state=42)

In [8]:
%%time

y_pred = muse_model.predict(X_test)

print("Classification report:\n")
print(classification_report(y_test, y_pred))

Classification report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       520
           1       1.00      1.00      1.00        39

    accuracy                           1.00       559
   macro avg       1.00      1.00      1.00       559
weighted avg       1.00      1.00      1.00       559

CPU times: total: 2min 13s
Wall time: 2min 17s
